# SpaceX Launch Sites Data Visualization

## Mark All Sites on a Map

### Importing Packages

In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

### Importing the data

In [4]:
spacex_df=pd.read_csv("data_falcon9.csv")
spacex_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,04-06-2010,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0003,-80.577366,28.561857,0
1,2,22-05-2012,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0005,-80.577366,28.561857,0
2,3,01-03-2013,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0007,-80.577366,28.561857,0
3,4,29-09-2013,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1,0,B1003,-120.610829,34.632093,0
4,5,03-12-2013,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B1004,-80.577366,28.561857,0


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


### Creating the Map

In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
# Initial the map
site_map = folium.Map(location=nasa_coordinate,zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index,row in launch_sites_df.iterrows():
    coordinate = [row['Latitude'],row['Longitude']]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['LaunchSite']))
    marker = folium.map.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

## Mark the success/failed launches for each site on the map

### Create markercluster object

In [7]:
marker_cluster = MarkerCluster()

### Create a new column in Launch_Site data frame to store marker color

In [8]:
launch_sites = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
markers = []
for index,record in launch_sites.iterrows(): 
    if record['Class']==1:
        markers.append('green')
    elif record['Class']==0:
        markers.append('red')
launch_sites['marker_color'] = markers
launch_sites.tail()

,LaunchSite,Latitude,Longitude,Class,marker_color
85,KSC LC 39A,28.608058,-80.603956,1,green
86,KSC LC 39A,28.608058,-80.603956,1,green
87,KSC LC 39A,28.608058,-80.603956,1,green
88,CCSFS SLC 40,28.561857,-80.577366,1,green
89,CCSFS SLC 40,28.561857,-80.577366,1,green


### For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [9]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in launch_sites.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    coordinate = [record['Latitude'],record['Longitude']]
    marker = folium.map.Marker(
    coordinate,
    icon=folium.Icon(color='white',icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map